# Project 5: Feature Selection and LASSO

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO. You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.
In the second part, you will implement your own LASSO solver, using coordinate descent.

We will continue to use the House data from Redfin.

In [1]:
import numpy as np
import pandas as pd

# Load in house sales data

In [279]:
df = pd.read_csv('merged.csv')

# Create new features

Recall in Project 2, we consider features that are some transformations of inputs.
Create square root of SQUARE FEET, square root of LOT SIZE, SQUARE of BEDS, and SQUARE of BATHS
* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.


In [3]:
from math import sqrt

In [280]:
df['sqft_sqrt'] = df['SQUARE FEET'].apply(lambda x:sqrt(x))
df['lot_sqrt'] = df['LOT SIZE'].apply(lambda x:sqrt(x))
df['bedrooms_square'] = (df['BEDS']) * (df['BEDS'])
df['bath_square'] = (df['BATHS']) * (df['BATHS'])

In [281]:
df.columns.values

array(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS',
       'BATHS', 'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT',
       'DAYS ON MARKET', '$/SQUARE FEET', 'HOA/MONTH', 'STATUS',
       'NEXT OPEN HOUSE START TIME', 'NEXT OPEN HOUSE END TIME',
       'URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)',
       'SOURCE', 'MLS#', 'FAVORITE', 'INTERESTED', 'LATITUDE',
       'LONGITUDE', 'sqft_sqrt', 'lot_sqrt', 'bedrooms_square',
       'bath_square'], dtype=object)

# Learn regression weights with L1 penalty


Let us fit a model with all the features available ('SQUARE FEET', 'LOT SIZE', 
                'BEDS', 'BATHS', 'CITY', 
                'PROPERTY TYPE', 'ZIP OR POSTAL CODE', 
                'YEAR BUILT'), plus the features we just created above. 
                
Note: For 'CITY' and 'PROPERTY TYPE' convert them to categorical features first then do label encoding.

In [11]:
from sklearn.preprocessing import LabelEncoder

In [14]:
# Transform city columns to categorical features

In [282]:
df['CITY'] = df.CITY.str.upper()

In [283]:
labelencoder = LabelEncoder()

In [284]:
df['CITY_cat'] = labelencoder.fit_transform(df['CITY'].astype(str))

In [285]:
# Transform property type to categorical data

In [286]:
df['PROPERTY TYPE_cat'] = labelencoder.fit_transform(df['PROPERTY TYPE'].astype(str))

In [287]:
df['ZIP OR POSTAL CODE'] = labelencoder.fit_transform(df['ZIP OR POSTAL CODE'])

In [288]:
all_features = ['SQUARE FEET', 'LOT SIZE', 
                'BEDS', 'BATHS', 'CITY_cat', 
                'PROPERTY TYPE_cat', 'ZIP OR POSTAL CODE', 
                'YEAR BUILT', 'sqft_sqrt','lot_sqrt','bedrooms_square','bath_square']

Applying L1 penalty = 1e7 using lasso Regression.

In [31]:
from sklearn.linear_model import Lasso

In [289]:
df2 = df[all_features+['PRICE']].dropna()

In [407]:
y = df2['PRICE']
X = df2[all_features]

In [408]:
model = Lasso(alpha=1e7)

In [409]:
model_1 = model.fit(X,y)

Find what features had non-zero weight.

In [410]:
def print_outcome(coefficients,intercept,input_features):
    for x in range(len(input_features)):
        print('The coefficient for {} is {}'.format(input_features[x],coefficients[x]))
    print('The intercept is {}'.format(intercept))

In [411]:
print_outcome(model_1.coef_,model_1.intercept_,all_features)

The coefficient for SQUARE FEET is 463.76165990713747
The coefficient for LOT SIZE is -0.11077069410463157
The coefficient for BEDS is 0.0
The coefficient for BATHS is 0.0
The coefficient for CITY_cat is -0.0
The coefficient for PROPERTY TYPE_cat is -0.0
The coefficient for ZIP OR POSTAL CODE is -0.0
The coefficient for YEAR BUILT is 0.0
The coefficient for sqft_sqrt is 0.0
The coefficient for lot_sqrt is -0.0
The coefficient for bedrooms_square is 0.0
The coefficient for bath_square is 0.0
The intercept is 221406.58328447246


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection.

According to this list of weights, which of the features have been chosen?

In [412]:
# From the previous test, I find that most of them's coefficients are 0.
# After removing those features, I only have SQUARE FEET, LOT SIZE, ZIP OR POSTAL CODE

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets with 45/45/10

In [413]:
from sklearn.model_selection import train_test_split

In [448]:
(train_valid,test) = train_test_split(df2,test_size=0.1)

In [449]:
(training,validation) = train_test_split(train_valid,test_size=0.5)

Next, we write a loop that does the following:
For l1_penalty in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^8] (to get this in Python, type np.logspace(1, 8, num=15).)
Fit a regression model with a given l1_penalty on TRAIN data. Specify alpha=l1_penalty in the parameter list.
Compute the RSS on VALIDATION data for that l1_penalty
Report which l1_penalty produced the lowest RSS on validation data.

In [450]:
training_X = training.drop(columns='PRICE')
training_y = training['PRICE']

In [451]:
validation_X = validation.drop(columns='PRICE')
validation_y = validation['PRICE']

In [452]:
l1_penalty_values = np.logspace(1, 8, num=15)

In [453]:
RSS = []
penalty = []
dataf = pd.DataFrame()
for l1_penalty in l1_penalty_values:
    model2 = Lasso(alpha = l1_penalty)
    model2_new = model2.fit(training_X,training_y)
    prediction = model2.predict(validation_X)
    error_square = (prediction - validation_y)**2
    sum_error_square = error_square.sum()
    RSS.append(sum_error_square)
    penalty.append(l1_penalty)
    print((l1_penalty,sum_error_square))
dataf['penalty'] = penalty
dataf['RSS'] = RSS

(10.0, 680072362506710.6)
(31.622776601683793, 680050359867750.2)
(100.0, 679981149290611.6)
(316.22776601683796, 679765964992598.2)
(1000.0, 679122280001194.5)
(3162.2776601683795, 677008680858271.4)
(10000.0, 670243965904292.0)
(31622.776601683792, 664400280165486.0)
(100000.0, 661078864492704.4)
(316227.7660168379, 657037385097791.0)
(1000000.0, 662315474719285.6)
(3162277.6601683795, 666379368131746.8)
(10000000.0, 667399161328158.1)
(31622776.60168379, 671157594958296.2)
(100000000.0, 688378449535456.8)


/Users/liziwei/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 206749533062321.03, tolerance: 58819556939.98183
  positive)
/Users/liziwei/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 170138211531970.44, tolerance: 58819556939.98183
  positive)
/Users/liziwei/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 101291337115957.72, tolerance: 58819556939.98183
  positive)
/Users/liziwei/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to incre

In [454]:
dataf

,penalty,RSS
0,1.000000e+01,6.800724e+14
1,3.162278e+01,6.800504e+14
2,1.000000e+02,6.799811e+14
3,3.162278e+02,6.797660e+14
4,1.000000e+03,6.791223e+14
5,3.162278e+03,6.770087e+14
6,1.000000e+04,6.702440e+14
7,3.162278e+04,6.644003e+14
8,1.000000e+05,6.610789e+14
9,3.162278e+05,6.570374e+14


What was the best value for the l1_penalty? and use this value of L1 penalty, how many nonzero weights do you have?

In [455]:
dataf.iloc[dataf['RSS'].idxmin()]

penalty    3.162278e+05
RSS        6.570374e+14
Name: 9, dtype: float64

In [437]:
# I find that the minimal RSS is when l1_penalty= 3.162278e+05.
# Then, use this l1_penalty to fit the model.

In [456]:
model3 = Lasso(alpha=3.162278e+05)

In [457]:
model3_new = model3.fit(X,y)

In [458]:
print_outcome(model3_new.coef_,model3_new.intercept_,all_features)

The coefficient for SQUARE FEET is 424.28852485257687
The coefficient for LOT SIZE is -0.4388651859567507
The coefficient for BEDS is 0.0
The coefficient for BATHS is 0.0
The coefficient for CITY_cat is -0.0
The coefficient for PROPERTY TYPE_cat is 0.0
The coefficient for ZIP OR POSTAL CODE is -7704.750049686064
The coefficient for YEAR BUILT is 4004.935516793208
The coefficient for sqft_sqrt is -0.0
The coefficient for lot_sqrt is 0.0
The coefficient for bedrooms_square is 0.0
The coefficient for bath_square is 0.0
The intercept is -7419034.349697219


In [459]:
np.count_nonzero(model3_new.coef_)

4

In [442]:
# Then, I found that the number of non-zero coefficients is more than the previous model.

# Limit the number of nonzero weights

In data industry, sometimes there are limitations to derive What if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them. Now let's consider the case where if we absolutely wanted to limit ourselves to, say, 7 features. 

In this section, you are going to implement a simple, two phase procedure to achive this goal:
Explore a large range of l1_penalty values to find a narrow region of l1_penalty values where models are likely to have the desired number of non-zero weights.
Further explore the narrow region you found to find a good value for l1_penalty that achieves the desired sparsity. Here, we will again use a validation set to choose the best value for l1_penalty.

In [443]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible l1_penalty_values:

In [460]:
l1_penalty_values = np.logspace(5, 6, num=20)

Now, implement a loop that search through this space of possible l1_penalty values:
For l1_penalty in np.logspace(5, 6, num=20):
Fit a regression model with a given l1_penalty on TRAIN data. Specify alpha=l1_penalty. in the parameter list.
Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.


In [461]:
non_zero = []
RSS = []
weights = []
dataf2 = pd.DataFrame()
for l1_penalty in l1_penalty_values:
    model4 = Lasso(alpha=l1_penalty)
    model4_new = model4.fit(training_X,training_y)
    number = np.count_nonzero(model4_new.coef_)
    predictions = model4_new.predict(training_X)
    square_error = (predictions - training_y) * (predictions - training_y)
    sum_square_error = square_error.sum()
    non_zero.append(number)
    RSS.append(sum_square_error)
    weights.append(l1_penalty)
    print("Current weight is:",l1_penalty)
    print("Current RSS:",sum_square_error)
    print("Number of non-zero is",number)
    print(" ")
dataf2['Weights'] = weights
dataf2['Non_Zero'] = non_zero
dataf2['RSS'] = RSS

Current weight is: 100000.0
Current RSS: 467366528224495.3
Number of non-zero is 7
 
Current weight is: 112883.78916846884
Current RSS: 467612918954086.75
Number of non-zero is 7
 
Current weight is: 127427.49857031347
Current RSS: 467926888301576.1
Number of non-zero is 7
 
Current weight is: 143844.9888287663
Current RSS: 468326972184190.56
Number of non-zero is 7
 
Current weight is: 162377.67391887208
Current RSS: 468836788527518.8
Number of non-zero is 7
 
Current weight is: 183298.07108324376
Current RSS: 469486434724871.4
Number of non-zero is 7
 
Current weight is: 206913.808111479
Current RSS: 470314262544727.5
Number of non-zero is 7
 
Current weight is: 233572.14690901214
Current RSS: 471369143914464.9
Number of non-zero is 7
 
Current weight is: 263665.08987303555
Current RSS: 472713352085270.4
Number of non-zero is 7
 
Current weight is: 297635.14416313195
Current RSS: 474426242861672.06
Number of non-zero is 7
 
Current weight is: 335981.8286283781
Current RSS: 4761039347

In [462]:
dataf2

,Weights,Non_Zero,RSS
0,100000.000000,7,4.673665e+14
1,112883.789168,7,4.676129e+14
2,127427.498570,7,4.679269e+14
3,143844.988829,7,4.683270e+14
4,162377.673919,7,4.688368e+14
5,183298.071083,7,4.694864e+14
6,206913.808111,7,4.703143e+14
7,233572.146909,7,4.713691e+14
8,263665.089873,7,4.727134e+14
9,297635.144163,7,4.744262e+14


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, can you try to code it up with at least two ways:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [463]:
l1_penalty_min = dataf2[dataf2['Non_Zero']>max_nonzeros].Weights.max()
l1_penalty_max = dataf2[dataf2['Non_Zero']<max_nonzeros].Weights.min()
print(l1_penalty_min)
print(l1_penalty_max)

nan
335981.8286283781


What values did you find for l1_penalty_min and l1_penalty_max, respectively?

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [134]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `alpah =l1_penalty` in the parameter list. 
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [137]:
penalty = []
non_zero = []
SSR = []
dataf3 = pd.DataFrame()
for l1_penalty in l1_penalty_values:
    model5 = Lasso(alpha = l1_penalty)
    model5_new = model5.fit(training_X,training_y)
    non = np.count_nonzero(model5_new.coef_)
    predictions = model5_new.predict(validation_X)
    error_square = (predictions - validation_y) * (predictions - validation_y)
    sum_square_error = error_square.sum()
    penalty.append(l1_penalty)
    non_zero.append(non)
    SSR.append(sum_square_error)
dataf3['Penalty'] = penalty
dataf3['Non_Zero'] = non_zero
dataf3['RSS'] = SSR
dataf3

,Penalty,Non_Zero,RSS
0,136068.282917,8,5.475356e+14
1,136341.149791,8,5.475437e+14
2,136614.016665,8,5.475518e+14
3,136886.883539,7,5.475600e+14
4,137159.750413,7,5.475684e+14
5,137432.617287,7,5.475769e+14
6,137705.484161,7,5.475854e+14
7,137978.351036,7,5.475939e+14
8,138251.217910,7,5.476026e+14
9,138524.084784,7,5.476112e+14


You can try another round of fine tuning of the parameters. The main technique to learn here is how to tune the parameters using granular way as described here.

What value of l1_penalty in our narrow range has the lowest RSS on the VALIDATION set and has sparsity equal to max_nonzeros?

What features in this model have non-zero coefficients?

In [142]:
dataf3.iloc[dataf3[dataf3['Non_Zero']==max_nonzeros].RSS.idxmin()]

Penalty     1.368869e+05
Non_Zero    7.000000e+00
RSS         5.475600e+14
Name: 3, dtype: float64

In [ ]:
# When the Non-zero is equal to 7, the lowest rss is 5.475600e+14, and its penalty 
# is 1.368869e+05.

In [144]:
model6 = Lasso(alpha=1.368869e+05)
model6_new = model6.fit(X,y)

In [145]:
print_outcome(model6_new.coef_,model6_new.intercept_,all_features)

The coefficient for SQUARE FEET is 363.37773163183556
The coefficient for LOT SIZE is -0.5914760576062719
The coefficient for BEDS is 0.0
The coefficient for BATHS is 0.0
The coefficient for CITY_cat is -7063.627495546559
The coefficient for PROPERTY TYPE_cat is 0.0
The coefficient for ZIP OR POSTAL CODE is -331.87300731386034
The coefficient for YEAR BUILT is 4665.145617623287
The coefficient for sqft_sqrt is -0.0
The coefficient for lot_sqrt is 0.0
The coefficient for bedrooms_square is 890.5467738197087
The coefficient for bath_square is 4715.595508991809
The intercept is 22652160.79698676


Does the non-zero coefficient makes sense to you? what does each of the parametere mean?

In [ ]:
# I think in some way, these non-zero coefficients make sense. However, I do not think
# that I should treat ZIP OR POSTAL CODE as a numerical type, because this column is 
# categorical data, and the number in this column may affect the final result.
# Therefore, I think this variable should be exclueded.

# Coordinate Descent for Lasso Regression

In this section, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

Let's reuse the get_num_data() and predict_output() function from Project 2. Copy them over.

In [146]:
def get_numpy_data(data, features, output):
    data['Constant'] = 1
    features = ['Constant'] + features
    feature_matrix = data[features].values
    output_array = data[output].values
    return(feature_matrix, output_array)

In [147]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix,weights)
    return(predictions)

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `SQUARE FEET` is very large overall compared to `BEDS`, for instance. As a result, weight for `SQUARE FEET` would be much smaller than weight for `BEDS`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features**: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [149]:
X = np.array([[1.,3.,9.],[4.,12.,19.]])
print (X)

[[ 1.  3.  9.]
 [ 4. 12. 19.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [150]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print (norms)

[ 4.12310563 12.36931688 21.02379604]


To normalize, apply element-wise division:

In [151]:
print (X / norms) # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[0.24253563 0.24253563 0.42808634]
 [0.9701425  0.9701425  0.90373784]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. We will use these norms to normalize the test data in the same way as we normalized the training data. 

In [162]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    features = feature_matrix / norms
    
    return(features,norms)

To test your function, run the following:

In [166]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print (features)
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print (norms)
# should print
# [5.  10.  15.]

[[0.6 0.6 0.6]
 [0.8 0.8 0.8]]
[ 5. 10. 15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [167]:
simple_features = ['SQUARE FEET', 'BEDS']
my_output = 'PRICE'
(simple_feature_matrix, output) = get_numpy_data(df2, simple_features, my_output)

Normalize the features first:

In [169]:
normal_simple_fea = normalize_features(simple_feature_matrix)

We assign some random set of initial weights and inspect the values of ro[i]:

In [170]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [187]:
predictions = predict_output(simple_feature_matrix,weights)

In [184]:
simple_feature_matrix.shape[]

3

Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [200]:
ro = []
for i in range(simple_feature_matrix.shape[1]):
    r1= (simple_feature_matrix[:,i]*(output - predictions + 
                                         (weights[i]*simple_feature_matrix[:,i]))).sum()
    ro.append(r1)
print(ro)

[1686990329.0, 3527203238868.0, 5874128372.0]


Whenever ro[i] falls between -l1_penalty/2 and l1_penalty/2, the corresponding weight w[i] is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of l1_penalty would not set w[1] zero, but would set w[2] to zero, if we were to take a step in that coordinate?

In [201]:
# For feature 1

In [203]:
2*ro[1]

7054406477736.0

In [204]:
# For feature 2

In [205]:
2*ro[2]

11748256744.0

In [ ]:
# If ro[i] > -lambda/2 and ro[i] < lambda/2, w[i]=0
# If ro[i] < -lambda/2 or ro[i] > lambda , w[i]=0.

In [120]:
# your code

-42182969.7294
-43606098.0185


What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

In [121]:
# your code

So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [344]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    predictions = predict_output(feature_matrix,weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = (feature_matrix[:,i]*(output - predictions + 
                                         (weights[i]*feature_matrix[:,i]))).sum()
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2:
        new_weight_i = ro_i+l1_penalty/2
    elif ro_i > l1_penalty/2:
        new_weight_i = ro_i-l1_penalty/2
    else:
        new_weight_i = 0
    
    return new_weight_i

To test the function, run the following cell:

In [342]:
import math

In [345]:
print (lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1))

0.4255588466910251


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [323]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    # your code #initialize weights
    weights = np.array(initial_weights)
    change_in_coordination = tolerance * np.ones(len(weights)) # define change_in_coordination
    while max(change_in_coordination) >= tolerance: 
        #for loop
        for i in range(len(weights)):
            old_weights_i = weights[i]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            change_in_coordination[i] = abs(old_weights_i - weights[i]) 
            
        
    return weights

Using the following parameters, learn the weights on the sales dataset. 

In [324]:
simple_features = ['SQUARE FEET', 'BEDS']
my_output = 'PRICE'
initial_weights = np.zeros(3)
l1_penalty = 5e6
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`.

In [304]:
(feature_matrix_2, output_2) = get_numpy_data(df2, simple_features, my_output)

In [305]:
(normalized_simple_feature_matrix,norms) = normalize_features(feature_matrix_2)

In [306]:
# your code
# normalize features

Then, run your implementation of LASSO coordinate descent:

In [307]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output_2, initial_weights, l1_penalty, tolerance)

In [308]:
weights

array([21424074.34219058, 22753616.26260392,        0.        ])

In [ ]:
# The weight for SQUARE FEET is 21424074.34219058.
# The weight for BEDS is 22753616.26260392.

***QUIZ QUESTIONS***
1. What is the RSS of the learned model on the normalized dataset? (Hint: use the normalized feature matrix when you make predictions.)
2. Which features had weight zero at convergence?

In [309]:
predictions2 = predict_output(normalized_simple_feature_matrix,weights)
square_error = (predictions2 - output_2) * (predictions2 - output_2)
RSS = square_error.sum()
RSS

1342535140154268.0

# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets using 80/20

In [346]:
(training,test) = train_test_split(df2,test_size=0.2)

Let's use all features as used previously.

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [349]:
all_features = ['SQUARE FEET', 'LOT SIZE', 
                'BEDS', 'BATHS', 'CITY_cat', 
                'PROPERTY TYPE_cat', 'ZIP OR POSTAL CODE', 
                'YEAR BUILT', 'sqft_sqrt','lot_sqrt','bedrooms_square','bath_square']
my_output = 'PRICE'

In [350]:
(feature_matrix_3, output_3) = get_numpy_data(training, all_features, my_output)

/Users/liziwei/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [351]:
(normalized_all_feature_matrix,norms_3) = normalize_features(feature_matrix_3)

In [352]:
# your code

First, learn the weights with `l1_penalty=1e6`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e6`, you will need them later.

In [353]:
l1_penalty=1e6
initial_weights = np.zeros(len(all_features)+1)
tolerance=1

In [355]:
weights1e6 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output_3, initial_weights, l1_penalty, tolerance)

In [361]:
print_outcome(weights1e6,weights1e6[len(all_features)],all_features)

The coefficient for SQUARE FEET is 19948574.259020492
The coefficient for LOT SIZE is 23952733.02246711
The coefficient for BEDS is 0.0
The coefficient for BATHS is 0.0
The coefficient for CITY_cat is 0.0
The coefficient for PROPERTY TYPE_cat is -3547168.3455931563
The coefficient for ZIP OR POSTAL CODE is 0.0
The coefficient for YEAR BUILT is -4706901.418796752
The coefficient for sqft_sqrt is 0.0
The coefficient for lot_sqrt is 0.0
The coefficient for bedrooms_square is 0.0
The coefficient for bath_square is 0.0
The intercept is 3420184.6384816905


In [ ]:
# I find that the BEDS, BATHS, CITY_cat, ZIP OR POSTAL CODE, sqft_sqrt, lot_sqrt,
# bedrooms_square and bath_square have 0 weights when penalty = 1e6.

What features had non-zero weight in this case?

Next, learn the weights with `l1_penalty=5e6`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights5e6`, you will need them later.

In [363]:
l1_penalty=5e6
initial_weights = np.zeros(len(all_features)+1)
tolerance=1

In [364]:
weights5e6 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output_3, initial_weights, l1_penalty, tolerance)

In [366]:
weights5e6

array([23327446.91713883, 10821464.71629212,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,  -422101.41552693,        0.        ,
              0.        ,        0.        ,        0.        ,
        5659183.48675774])

In [365]:
print_outcome(weights5e6,weights5e6[len(all_features)],all_features)

The coefficient for SQUARE FEET is 23327446.917138826
The coefficient for LOT SIZE is 10821464.716292117
The coefficient for BEDS is 0.0
The coefficient for BATHS is 0.0
The coefficient for CITY_cat is 0.0
The coefficient for PROPERTY TYPE_cat is 0.0
The coefficient for ZIP OR POSTAL CODE is 0.0
The coefficient for YEAR BUILT is -422101.41552693024
The coefficient for sqft_sqrt is 0.0
The coefficient for lot_sqrt is 0.0
The coefficient for bedrooms_square is 0.0
The coefficient for bath_square is 0.0
The intercept is 5659183.486757736


In [367]:
# I find that the BEDS, BATHS, CITY_cat, PROPERTY TYPE_cat,
# ZIP OR POSTAL CODE, sqft_sqrt, lot_sqrt,
# bedrooms_square and bath_square have 0 weights when penalty = 5e6.

What features had non-zero weight in this case?

Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=1e2`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [368]:
l1_penalty=1e4
initial_weights = np.zeros(len(all_features)+1)
tolerance=1e2

In [369]:
weights1e4 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output_3, initial_weights, l1_penalty, tolerance)

In [370]:
print_outcome(weights1e4,weights1e4[len(all_features)],all_features)

The coefficient for SQUARE FEET is -296119560.5260999
The coefficient for LOT SIZE is 37793724.92172788
The coefficient for BEDS is -1075505.8672239594
The coefficient for BATHS is 2869645.003249021
The coefficient for CITY_cat is 6813803.634347554
The coefficient for PROPERTY TYPE_cat is -5365627.589544139
The coefficient for ZIP OR POSTAL CODE is 725621.0642663033
The coefficient for YEAR BUILT is -7032823.134882799
The coefficient for sqft_sqrt is 326987258.9888274
The coefficient for lot_sqrt is -27476626.754277505
The coefficient for bedrooms_square is 2391378.5429834533
The coefficient for bath_square is 714261.5400839491
The intercept is -3174135.635059706


In [371]:
weights1e4

array([-2.96119561e+08,  3.77937249e+07, -1.07550587e+06,  2.86964500e+06,
        6.81380363e+06, -5.36562759e+06,  7.25621064e+05, -7.03282313e+06,
        3.26987259e+08, -2.74766268e+07,  2.39137854e+06,  7.14261540e+05,
       -3.17413564e+06])

In [372]:
# I find that all features have non-zero weights when penalty = 1e4 and tolerance = 1e2.

What features had non-zero weight in this case?

## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

In [373]:
(feature_matrix, output) = get_numpy_data(training, all_features, my_output)

/Users/liziwei/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [375]:
(features, norms) = normalize_features(feature_matrix)

Create a normalized version of each of the weights learned above. (`weights1e6`, `weights5e6`, `weights1e4`).

In [377]:
weights_normalized1e6 = weights1e6 / norms

In [378]:
weights_normalized5e6 = weights5e6 / norms

In [379]:
weights_normalized1e4 = weights1e4 / norms

To check your results, if you call `normalized_weights1e6` the normalized version of `weights1e6`, should return 11.22951655151892

In [192]:
weights1e6_normalized[2]

11.22951655151892

## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [385]:
(feature_matrix_test, output_test) = get_numpy_data(test, all_features, my_output)

/Users/liziwei/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [386]:
# for penalty = 1e6

In [387]:
prediction1 = predict_output(feature_matrix_test,weights_normalized1e6 )
error_1_square = (prediction1-output_test) * (prediction1-output_test)
RSS_1 = error_1_square.sum()
print(RSS_1)

264852018130349.75


In [ ]:
# for penalty = 5e6

In [388]:
prediction2 = predict_output(feature_matrix_test,weights_normalized5e6 )
error_2_square = (prediction2-output_test) * (prediction2-output_test)
RSS_2 = error_2_square.sum()
print(RSS_2)

275228248833321.66


In [ ]:
# for penalty = 1e4

In [389]:
prediction3 = predict_output(feature_matrix_test,weights_normalized1e4 )
error_3_square = (prediction3-output_test) * (prediction3-output_test)
RSS_3 = error_3_square.sum()
print(RSS_3)

268359048168217.5


Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

Which model performed best on the test data? What is your interpretation on the results?

In [ ]:
# Based on the previous results, I find that when penalty equals to 1e6, the 
# RSS is the lowest compared to the rest. 
# Therefore, I think that the penalty 1e6 performs the best.